In [ ]:
import pandas as pd

time_group = pd.read_csv("/Volumes/data_files/UKB_data/processed_data/sle_group.csv")

within_five = time_group[time_group["Group"]=="0-5"]
within_five_eid = within_five["eid"].tolist()

five_ten = time_group[time_group["Group"]=="5-10"]
five_ten_eid = five_ten["eid"].tolist()

ten_fifteen = time_group[time_group["Group"]=="10-15"]
ten_fifteen_eid = ten_fifteen["eid"].tolist()


In [ ]:
immune_basic = pd.read_csv("/Volumes/data_files/UKB_data/processed_data/immune_basic_fp.csv")
pro = pd.read_csv("/Volumes/data_files/UKB_data/processed_data/pro_sler")
st_features = pd.read_csv("/Volumes/data_files/UKB_data/processed_data/sle_student_t_features.csv")

In [ ]:
immune_pro = pro.merge(immune_basic, on='eid', how='left')
immune_pro = immune_pro.merge(st_features[['eid', 'alcohol_amount']], on='eid', how='left')

follow_up = ['fp-SLE', 'fp-RA', 'fp-SS', 'fp-Systemic Sclerosis', 'fp-APS', 'fp-Autoimmune thyroiditis']
srd = ['srd_SLE', 'srd_RA','srd_SS','srd_Systemic Sclerosis', 'srd_APS', 'srd_Autoimmune thyroiditis']
disease = ['SLE','RA','SS', 'Systemic Sclerosis', 'APS', 'Autoimmune thyroiditis']

In [ ]:
print(immune_pro.columns.tolist())

In [ ]:
unuse_col = ['RA', 'SS', 'Systemic Sclerosis', 'APS', 'Autoimmune thyroiditis', 'icd10_SLE_dates', 'icd10_RA_dates', 'icd10_SS_dates', 'icd10_Systemic Sclerosis_dates', 'icd10_APS_dates', 'icd10_Autoimmune thyroiditis_dates', 'srd_RA', 'srd_SS', 'srd_Systemic Sclerosis', 'srd_APS', 'srd_Autoimmune thyroiditis', 'bmi', 'date_attend', 'birth_year', 'birth_month', 'fp-len', 'fp-RA', 'fp-SS', 'fp-Systemic Sclerosis', 'fp-APS', 'fp-Autoimmune thyroiditis', 'age']
immune_pro.drop(columns=unuse_col, inplace=True)
print(immune_pro.columns.tolist())

In [ ]:
# sle_pro = immune_pro[immune_pro["srd_SLE"].isna()]
sle_pro = immune_pro[immune_pro["fp-SLE"] > 0]
sle_pro.drop(["srd_SLE"], axis=1, inplace=True)
sle_pro = sle_pro.fillna(sle_pro.median(numeric_only=True))

sle_pro_5 = sle_pro[sle_pro["eid"].isin(within_five_eid)]
sle_pro_10 = sle_pro[sle_pro["eid"].isin(five_ten_eid)]
sle_pro_15 = sle_pro[sle_pro["eid"].isin(ten_fifteen_eid)]
sle_control = sle_pro[sle_pro["SLE"]==0]
print(sle_pro_5.shape,sle_pro_10.shape,sle_pro_15.shape,sle_control.shape)

sle_pro_5_cox = pd.concat([sle_pro_5, sle_control], ignore_index=True)
sle_pro_10_cox = pd.concat([sle_pro_10, sle_control], ignore_index=True)
sle_pro_15_cox = pd.concat([sle_pro_15, sle_control], ignore_index=True)

print(sle_pro_5_cox.shape,sle_pro_10_cox.shape,sle_pro_15_cox.shape)

In [ ]:
sle_pro_15_cox

In [ ]:
pro_cols = [col for col in sle_pro.columns if col not in ["eid", "sex", "ethnicity", "alcohol_amount", "SLE","fp-SLE"]]
print(pro_cols)

In [ ]:
# covar_cols = ['sex', 'ethnicity', 'alcohol_amount']
covar_cols = ['sex', 'ethnicity']

In [ ]:
import pandas as pd
from lifelines import CoxPHFitter
from tqdm import tqdm
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.multitest import fdrcorrection
from mne.stats import bonferroni_correction

from scipy.stats import norm
import numpy as np

def cox_cal(cox_data, out_file):
    
    # 初始化输出结果数据框
    results_all = []    
    n_tests = len(pro_cols)
    
    # 遍历每个特征列
    for t, t_pro in enumerate(tqdm(pro_cols, desc="Processing features")):
        try:
            # 构建用于分析的列
            t_col = ["SLE","fp-SLE",t_pro] + covar_cols
            t_ra_pro = cox_data[t_col].copy()  # 复制数据，避免警告
            # print(t_col)
    
            # 检查并删除缺失值
            # t_ra_pro = t_ra_pro.dropna(subset=[t_pro])
            t_ra_pro.rename(columns={t_pro: "target_pro"}, inplace=True)
    
            # 初始化Cox模型并拟合数据
            cph = CoxPHFitter()
            cph.fit(t_ra_pro, duration_col="fp-SLE", event_col="SLE", formula=" + ".join(['target_pro'] + covar_cols))
    
            # 提取模型结果
            hr = cph.hazard_ratios_.get('target_pro', None)
            # print(hr)
    
            # 提取置信区间，带检查
            # print(cph.confidence_intervals_)
            if '95% lower-bound' in cph.confidence_intervals_.columns:
                conf_int = cph.confidence_intervals_.loc['target_pro']
                lbd, ubd = conf_int['95% lower-bound'], conf_int['95% upper-bound']
            else:
                lbd, ubd = None, None
    
            # 提取 p 值
            # print(cph.summary)
            pval = cph.summary.loc['target_pro', 'p']
    
            # 保存结果
            results_all.append({'Feature': t_pro, 'HR': hr, 'Lower CI': lbd, 'Upper CI': ubd, 'p-value': pval})
            # print(f"Processed feature '{t_pro}' with HR = {hr:.3f}, p = {pval:.3f}")
        except Exception as e:
            print(f"Error processing feature '{t_pro}': {e}")


    results_all_df = pd.DataFrame(results_all)


    # results_df = pd.DataFrame(results)
    # results_df["Bonferroni"] = results_df["p-value"].apply(lambda p: min(p * n_tests, 1))
    # # 使用 Benjamini-Hochberg 方法进行 FDR 校正
    # results_df["FDR"] = multipletests(results_df["p-value"], method="fdr_bh")[1]

    # FDR correction
    _, p_f_fdr = fdrcorrection(results_all_df['p-value'].fillna(1))
    
    # Bonferroni correction
    alpha = 0.05
    p_f_bfi = results_all_df['p-value'].fillna(1) * len(results_all_df)  # Bonferroni correction
    p_f_bfi = p_f_bfi.clip(upper=alpha)  # Ensure p-values do not exceed alpha level

    
    results_all_df["Bonferroni"] = p_f_bfi
    # 使用 Benjamini-Hochberg 方法进行 FDR 校正
    results_all_df["FDR"] = p_f_fdr
    results_all_df.to_csv(out_file, index=False)


In [ ]:
cox_cal(sle_pro, "/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_cox_2.csv")
# cox_cal(sle_pro_5_cox, "/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_5_cox.csv")
# cox_cal(sle_pro_10_cox, "/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_10_cox.csv")
# cox_cal(sle_pro_15_cox, "/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_15_cox.csv")

In [21]:
import pandas as pd
from lifelines import CoxPHFitter
from tqdm import tqdm
from statsmodels.stats.multitest import fdrcorrection
import numpy as np

def cox_cal(cox_data, out_file, covar_cols, pro_cols):
    # 初始化输出结果数据框
    results_all = []
    n_tests = len(pro_cols)

    # 遍历每个特征列
    for t, t_pro in enumerate(tqdm(pro_cols, desc="Processing features")):
        try:
            # 构建用于分析的列
            t_col = ["SLE", "fp-SLE", t_pro] + covar_cols
            t_ra_pro = cox_data[t_col].copy()  # 复制数据，避免警告
            t_ra_pro.rename(columns={t_pro: "target_pro"}, inplace=True)

            # 计算事件发生和未发生的比例，用来设置样本权重
            event_count = t_ra_pro['SLE'].sum()  # 事件发生的数量
            non_event_count = len(t_ra_pro) - event_count  # 事件未发生的数量
            event_weight = non_event_count / event_count  # 事件发生组的权重
            non_event_weight = 1  # 事件未发生组的权重

            # 给每一行样本分配权重
            t_ra_pro['weights'] = np.where(t_ra_pro['SLE'] == 1, event_weight, non_event_weight)

            # 初始化Cox模型并拟合数据，使用加权列
            cph = CoxPHFitter()
            cph.fit(t_ra_pro, duration_col="fp-SLE", event_col="SLE", formula=" + ".join(['target_pro'] + covar_cols), weights_col='weights')

            # 提取模型结果
            hr = cph.hazard_ratios_.get('target_pro', None)

            # 提取置信区间
            if '95% lower-bound' in cph.confidence_intervals_.columns:
                conf_int = cph.confidence_intervals_.loc['target_pro']
                lbd, ubd = conf_int['95% lower-bound'], conf_int['95% upper-bound']
            else:
                lbd, ubd = None, None

            # 提取 p 值
            pval = cph.summary.loc['target_pro', 'p']

            # 保存结果
            results_all.append({'Feature': t_pro, 'HR': hr, 'Lower CI': lbd, 'Upper CI': ubd, 'p-value': pval})
        except Exception as e:
            print(f"Error processing feature '{t_pro}': {e}")

    # 将所有结果保存为DataFrame
    results_all_df = pd.DataFrame(results_all)

    # FDR correction
    _, p_f_fdr = fdrcorrection(results_all_df['p-value'].fillna(1))

    # Bonferroni correction
    alpha = 0.05
    p_f_bfi = results_all_df['p-value'].fillna(1) * len(results_all_df)  # Bonferroni correction
    p_f_bfi = p_f_bfi.clip(upper=alpha)  # Ensure p-values do not exceed alpha level

    # 将修正后的p值添加到结果中
    results_all_df["Bonferroni"] = p_f_bfi
    results_all_df["FDR"] = p_f_fdr

    # 将结果保存到CSV文件
    results_all_df.to_csv(out_file, index=False)

# 示例调用
# 假设你已经定义了 `cox_data`, `covar_cols`, 和 `pro_cols` 等必要变量
# cox_cal(cox_data, "output_results.csv", covar_cols, pro_cols)


In [ ]:
cox_cal(sle_pro, "/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_cox_weighted.csv",covar_cols, pro_cols)

In [24]:
check = pd.read_csv("/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_cox_weighted.csv")
check = check[check["Bonferroni"]<0.05]
check

,Feature,HR,Lower CI,Upper CI,p-value,Bonferroni,FDR
0,a1bg,1.472747,0.338240,0.436020,2.551376e-54,7.457673e-51,5.237130e-54
1,aamdc,1.069172,0.049818,0.083952,1.577652e-14,4.611478e-11,2.137913e-14
2,aarsd1,1.093932,0.076243,0.103314,1.217038e-38,3.557403e-35,2.198642e-38
3,abca2,1.169349,0.130764,0.182131,7.418133e-33,2.168320e-29,1.256269e-32
4,abhd14b,1.105198,0.086521,0.113528,9.344369e-48,2.731359e-44,1.824555e-47
...,...,...,...,...,...,...,...
2916,znf75d,1.044212,0.028812,0.057712,4.415446e-09,1.290635e-05,5.539205e-09
2917,znf830,0.853174,-0.171602,-0.145982,2.147626e-130,6.277511e-127,6.808580e-130
2919,znrf4,1.056684,0.038912,0.071360,2.725355e-11,7.966212e-08,3.542113e-11
2920,zp3,0.985963,-0.017504,-0.010770,1.870478e-16,5.467407e-13,2.606009e-16


In [25]:
import pandas as pd
from lifelines import CoxPHFitter
from tqdm import tqdm
from statsmodels.stats.multitest import fdrcorrection
import numpy as np

def cox_cal(cox_data, out_file, covar_cols, pro_cols):
    # 初始化输出结果数据框
    results_all = []
    n_tests = len(pro_cols)

    # 遍历每个特征列
    for t, t_pro in enumerate(tqdm(pro_cols, desc="Processing features")):
        try:
            # 构建用于分析的列
            t_col = ["SLE", "fp-SLE", t_pro] + covar_cols
            t_ra_pro = cox_data[t_col].copy()  # 复制数据，避免警告
            t_ra_pro.rename(columns={t_pro: "target_pro"}, inplace=True)

            # 手动设置权重：事件发生组的权重为 10，未发生事件组的权重为 1
            t_ra_pro['weights'] = np.where(t_ra_pro['SLE'] == 1, 10, 1)

            # 初始化Cox模型并拟合数据，使用加权列
            cph = CoxPHFitter()
            cph.fit(t_ra_pro, duration_col="fp-SLE", event_col="SLE", formula=" + ".join(['target_pro'] + covar_cols), weights_col='weights')

            # 提取模型结果
            hr = cph.hazard_ratios_.get('target_pro', None)

            # 提取置信区间
            if '95% lower-bound' in cph.confidence_intervals_.columns:
                conf_int = cph.confidence_intervals_.loc['target_pro']
                lbd, ubd = conf_int['95% lower-bound'], conf_int['95% upper-bound']
            else:
                lbd, ubd = None, None

            # 提取 p 值
            pval = cph.summary.loc['target_pro', 'p']

            # 保存结果
            results_all.append({'Feature': t_pro, 'HR': hr, 'Lower CI': lbd, 'Upper CI': ubd, 'p-value': pval})
        except Exception as e:
            print(f"Error processing feature '{t_pro}': {e}")

    # 将所有结果保存为DataFrame
    results_all_df = pd.DataFrame(results_all)

    # FDR correction
    _, p_f_fdr = fdrcorrection(results_all_df['p-value'].fillna(1))

    # Bonferroni correction
    alpha = 0.05
    p_f_bfi = results_all_df['p-value'].fillna(1) * len(results_all_df)  # Bonferroni correction
    p_f_bfi = p_f_bfi.clip(upper=alpha)  # Ensure p-values do not exceed alpha level

    # 将修正后的p值添加到结果中
    results_all_df["Bonferroni"] = p_f_bfi
    results_all_df["FDR"] = p_f_fdr

    # 将结果保存到CSV文件
    # results_all_df.to_csv(out_file, index=False)
    return results_all_df
# 示例调用
# 假设你已经定义了 `cox_data`, `covar_cols`, 和 `pro_cols` 等必要变量
# cox_cal(cox_data, "output_results.csv", covar_cols, pro_cols)

In [ ]:
check = cox_cal(sle_pro, "/Volumes/data_files/UKB_data/immune_result/cox3/sle_pro_cox_weighted.csv",covar_cols, pro_cols)

In [27]:
check = check[check["Bonferroni"]<0.05]
check

,Feature,HR,Lower CI,Upper CI,p-value,Bonferroni,FDR
5,abl1,1.143917,0.087458,0.181458,2.057780e-08,6.014890e-05,4.800391e-08
8,acaa1,1.133344,0.085746,0.164599,4.889682e-10,1.429254e-06,1.242830e-09
13,ace2,1.280796,0.189195,0.305768,8.660876e-17,2.531574e-13,2.916560e-16
14,ache,0.659033,-0.540270,-0.293693,3.381739e-11,9.884824e-08,9.043755e-11
16,acox1,1.168501,0.091378,0.220066,2.101873e-06,6.143774e-03,4.269475e-06
...,...,...,...,...,...,...,...
2909,zbp1,1.772683,0.515658,0.629330,9.358366e-87,2.735450e-83,1.144540e-85
2911,zbtb17,1.747797,0.507757,0.608955,9.796176e-104,2.863422e-100,1.424588e-102
2912,zcchc8,1.154123,0.086808,0.199873,6.710434e-07,1.961460e-03,1.412138e-06
2917,znf830,0.810950,-0.266918,-0.152181,8.116194e-13,2.372364e-09,2.353535e-12
